In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ica import encode, prbs, qpsk, qam16, noise

In [ ]:
N, M = 2, int(4e+3) # 信号数，サンプル数 (4000ポイント)
snr = 40 # dB 原信号レベルに対して -40 dB の雑音レベルを指定
s = np.array([encode(prbs(int(M)),qam16) for i in range(N)]) # 16 QAM で分散 1，M サンプルの信号列を N 本生成する
a = np.random.rand(s.shape[0],s.shape[0]) # N (=　S の行の数) x N の混合行列を作る
a[0,0] = 0
a[1,1] = 0
a = np.exp(1j * 2 * np.pi * a) # 簡単のため長さ 1 の混合ベクトルに限定する
n = noise((a.shape[0], s.shape[-1])) / 10**(snr/20) # 雑音を生成
x = np.dot(a, s) + n # 観測信号を生成

calcm = lambda m12,m21: np.array([[0,m12],[m21,0]])
f = lambda x: (x.conj() * x) * x
g = lambda x: x.conj()
mu = 0.01
dm = lambda y1, y2: mu * f(y1) * g(y2)

m12, m21 = np.exp(1j*2*np.pi*0.25), np.exp(1j*2*np.pi*0.25) # 初期状態
m = []
e = []
for xi in x.T:
    y = np.dot(np.linalg.inv(np.eye(2,2) + calcm(m12,m21)), xi)
    m12 += dm(y[0], y[1])
    m21 += dm(y[1], y[0])
    m.append((m12,m21))
    y = np.dot(np.linalg.inv(np.eye(2,2) + calcm(m12,m21)), x)
    e.append(np.dot(f(y[0,:]), g(y[1,:])).sum()/s.shape[1])
    
# 計算結果を出力する
    
z = x[0,:]
plt.plot(np.real(z), np.imag(z), '.', markersize=1)
z = x[1,:]
plt.plot(np.real(z), np.imag(z), '.', markersize=1)
plt.axis('square')
plt.show()

y = np.dot(np.linalg.inv(np.eye(2,2) + calcm(m12,m21)), x)
plt.plot(np.real(y[0,:]), np.imag(y[0,:]), '.', markersize=1)
plt.plot(np.real(y[1,:]), np.imag(y[1,:]), '.', markersize=1)
plt.axis('square')
plt.show()

m = np.array(m)
plt.plot(np.real(m[:,0]),np.imag(m[:,0]))
plt.plot(np.real(m[:,1]),np.imag(m[:,1]))
plt.show()

base = np.abs(np.dot(f(s[0,:]), g(s[1,:])).sum()/s.shape[1])
e = np.array(e)
plt.plot(np.abs(e))
plt.plot([0,len(e)],[base,base])
plt.show()